In [1]:
# INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
#
# Copyright © 2020-2021 Intel Corporation.
#
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy,
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are
# expressly stated in the License.

# Output Spike Streamer
This tutorial shows how to use the outputSpikeStreamer and builds on the inputSpikeStreamer tutorial. 

The outputSpikeStreamer returns accumulated spike counts from a group of output neurons at a user specified interval. To use the outputSpikeStreamer, the user needs to:
 - Specify the interval at which to report accumulated spike counts (high intervals mean less off-chip communication, resulting in higher speed)
 - Specify which output neurons to monitor for spikes.

This tutorial will have 3 parts:
 - an inputSpikeStreamer to send spikes to Loihi
 - an identity connection between the inputSpikeStreamer and a second group of neurons
 - an outputSpikeStreamer monitoring and reporting output spikes form the second group of neurons 

In [2]:
import numpy as np
import nxsdk.api.n2a as nx
from nxsdk_modules.spike_streamer.src.streaming import SpikeStreamer

# Create the SpikeStreamer

In [3]:
spikesPerPacket = 1024 # larger packets are more efficient, but add latency when very few spikes are present
microsecondsPerTimestep = 1000 # 1ms is typical for sensor interaction, but much smaller timesteps are possible if needed
numInputNeurons = 10 # how many input neurons?
runTime = 1000 #10 seconds if microsecondsPerTimestep is 1000

net = nx.NxNet()
# create the spike streamer
spikeStreamer = SpikeStreamer(net)
spikeStreamer.setupSpikeInput(numInputNeurons, spikesPerPacket, microsecondsPerTimestep)

# Setup some output neurons for use with the SpikeStreamer
We'll just create a one-to-one connection from input to output (identity)

In [4]:
# replace this with your own network
compProto = nx.CompartmentPrototype(vThMant=1,
                                    compartmentVoltageDecay = 4095,
                                    compartmentCurrentDecay = 4095)
myNetwork = net.createCompartmentGroup(size=numInputNeurons, prototype=compProto)

# make the connection from spikeStreamer to your network. Modify the connectivity and connection properties here
connProto = nx.ConnectionPrototype(weight=2)
spikeStreamer.inputLayer.connect(myNetwork, prototype=connProto, connectionMask=np.identity(numInputNeurons))

# Create the outputSpikeStreamer


In [5]:
accumulationInterval = 100 # report spike counts after every 100 Loihi timesteps
spikeStreamer.setupSpikeOutput(myNetwork, accumulationInterval)

# Compile the board and setup the inputSpikeStreamer and outputSpikeStreamer
This configures the snips used by the inputSpikeStreamer and outputSpikeStreamer

In [6]:
# compile the board
compiler = nx.N2Compiler()
board = compiler.compile(net)
spikeStreamer.configureStreamer(board)
board.run(runTime, aSync=True)

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.69:46387
INFO:DRV:      Host server up..............Done 0.18s
INFO:DRV:      Compiling Embedded snips....Done 0.53s
INFO:DRV:      Encoding axons/synapses.....Done 6.60ms
INFO:DRV:      Encoding probes.............Done 0.77ms
INFO:HST:  Args chip=0 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1583951613.5867388/launcher_chip0_lmt0.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=1 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=2 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=3 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Args chip=4 cpu=0 /home/gorchard/nxsdk-nxsdk/nxsdk/driver/../bin/arm/inactive_chips.bin --chips=1

In [7]:
# Send the spike data
"""
# for a real-time demo, this would look something like:
while(True): #some condition to determine when the demo has ended
    spikeTimes, spikeTargets = getSpikesFromSkin()
    spikeStreamer.sendSpikes(spikeTargets, spikeTimes)
"""

# for this artificial test, we'll fake some spike data
maxSpikesPerPacket = 50 # only used for our test/artificial spike generation. In real-time demo the number of spikes isn't limited
numSendIterations = runTime//accumulationInterval # lets assume we send spikes 10 different times.
minTime = 0
allTimes = np.zeros((0,), dtype=int)
allTargets = np.zeros((0,), dtype=int)

# create a variable to hold all the results
results = np.zeros((spikeStreamer.numOutputs,numSendIterations), dtype=np.int)

for ii in range(numSendIterations):
    maxTime = (ii+1)*runTime//numSendIterations
    
    # randomly generate spikes. Column 0 is neuron, column 1 is time.
    spikes = np.zeros((maxSpikesPerPacket,2), dtype=int)
    spikes[:,0] = np.random.randint(0, numInputNeurons, size=(maxSpikesPerPacket,), dtype=int)
    spikes[:,1] = np.random.randint(minTime, maxTime, size=(maxSpikesPerPacket,), dtype=int)
    spikes = np.unique(spikes, axis=0) # we only want 1 spike per neuron per timestep
    
    minTime = maxTime
    
    # sort into chronological order (column 1)
    spikes = spikes[spikes[:,1].argsort(), :]

    # record the spikes so we can verify them later
    allTimes = np.concatenate((allTimes, spikes[:,1]))
    allTargets = np.concatenate((allTargets, spikes[:,0]))

    # send the spikes
    spikeStreamer.sendSpikes(spikes[:,0], spikes[:,1])
    
    # get the latest results. For this to work, the chip must have advanced past the time at which this result is expected.
    # For example, if the accumulationInterval==100, then the first results will only be reported after the Loihi timestep passes 100
    spikeStreamer.advanceTime(maxTime)
    results[:,ii] = spikeStreamer.getResults()

INFO:HST:  chip=2 cpu=0 halted, status=0x0
INFO:HST:  chip=3 cpu=0 halted, status=0x0
INFO:HST:  chip=4 cpu=0 halted, status=0x0
INFO:HST:  chip=5 cpu=0 halted, status=0x0
INFO:HST:  chip=6 cpu=0 halted, status=0x0
INFO:HST:  chip=7 cpu=0 halted, status=0x0
INFO:HST:  chip=8 cpu=0 halted, status=0x0
INFO:HST:  chip=9 cpu=0 halted, status=0x0
INFO:HST:  chip=10 cpu=0 halted, status=0x0
INFO:HST:  chip=11 cpu=0 halted, status=0x0
INFO:HST:  chip=12 cpu=0 halted, status=0x0
INFO:HST:  chip=13 cpu=0 halted, status=0x0
INFO:HST:  chip=14 cpu=0 halted, status=0x0
INFO:HST:  chip=15 cpu=0 halted, status=0x0
INFO:HST:  chip=16 cpu=0 halted, status=0x0
INFO:HST:  chip=17 cpu=0 halted, status=0x0
INFO:HST:  chip=18 cpu=0 halted, status=0x0
INFO:HST:  chip=19 cpu=0 halted, status=0x0
INFO:HST:  chip=20 cpu=0 halted, status=0x0
INFO:HST:  chip=21 cpu=0 halted, status=0x0
INFO:HST:  chip=22 cpu=0 halted, status=0x0
INFO:HST:  chip=23 cpu=0 halted, status=0x0
INFO:HST:  chip=24 cpu=0 halted, status=

In [8]:
# at any point, "flushSpikes" can be called to flush any buffered spikes to the chip
#spikeInputStreamer.flushSpikes()

In [9]:
# before finishing the run, make sure to advance the chip time to the end of the run
spikeStreamer.advanceTime(runTime)

In [10]:
board.finishRun()
board.disconnect()

INFO:DRV:      Executing...................Done 2.83ms
INFO:DRV:      Processing timeseries.......Done 1.92ms
INFO:HST:  chip=0 cpu=0 halted, status=0x0


In [11]:
# check that the spikes received from probes match the spikes we injected

# How many timesteps delay between input and output 
# 2 because input layer incurs 1 timestep delay and our network incurs another timestep delay
timeOffset=2

for jj in range(spikeStreamer.numOutputs):
    thisTimes = allTimes[allTargets==jj] + timeOffset
    for ii in range(numSendIterations):
        assert results[jj, ii] == np.sum(np.logical_and((ii*accumulationInterval)<=thisTimes, thisTimes<((ii+1)*accumulationInterval))), "error, spike mismatch"
print("Success!!!")

Success!!!
